In [7]:
import requests
from bs4 import BeautifulSoup

# URL der Schweizer Super League Übersicht (Wettbewerb C1)
url = "https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Suche nach der Tabelle mit den Club-Daten
table = soup.find("table", {"class": "items"})
if table:
    print("Tabelle gefunden!")
    tbody = table.find("tbody")
    if tbody:
        rows = tbody.find_all("tr")
        print(f"Anzahl der Zeilen: {len(rows)}")
        if rows:
            # Ausgabe der ersten Zeile zur Kontrolle
            print(rows[0].get_text(separator=" | "))
    else:
        print("Kein <tbody> gefunden!")
else:
    print("Keine Tabelle gefunden!")


Tabelle gefunden!
Anzahl der Zeilen: 12

 | FC Basel 1893 |   | 31 | 23,9 | 19 | 2,14 Mio. € | 66,20 Mio. €


In [17]:
import asyncio
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import json

async def get_matchday_data():
    matchday_url = "https://www.transfermarkt.ch/super-league/spieltag/wettbewerb/C1/saison_id/2023/page/1"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    }
    asession = AsyncHTMLSession()
    response = await asession.get(matchday_url, headers=headers)
    # Rendern der Seite, um den JavaScript-Inhalt zu laden
    await response.html.arender(timeout=20)
    
    soup = BeautifulSoup(response.html.html, "html.parser")
    
    matchday_table = soup.find("table", {"class": "items"})
    match_data = []
    if matchday_table:
        tbody = matchday_table.find("tbody")
        if tbody:
            rows = tbody.find_all("tr")
            for row in rows:
                cells = row.find_all("td")
                if len(cells) >= 10:
                    data = {
                        "position": cells[0].get_text(strip=True),
                        "club_name": cells[1].get_text(strip=True),
                        "played": cells[2].get_text(strip=True),
                        "won": cells[3].get_text(strip=True),
                        "drawn": cells[4].get_text(strip=True),
                        "lost": cells[5].get_text(strip=True),
                        "goals_for": cells[6].get_text(strip=True),
                        "goals_against": cells[7].get_text(strip=True),
                        "goal_difference": cells[8].get_text(strip=True),
                        "points": cells[9].get_text(strip=True)
                    }
                    match_data.append(data)
    else:
        print("Keine Tabelle mit Klasse 'items' gefunden!")
    
    return match_data

# In einem Notebook kannst du asynchron direkt await nutzen:
match_data = await get_matchday_data()
print(json.dumps(match_data, indent=2, ensure_ascii=False))


AttributeError: module 'websockets' has no attribute 'client'